## Goal
Pack 14 Christmas tree polygons into the smallest square. Target score: 0.3696

Score formula: `max(width, height)^2 / N`

In [1]:
!git clone https://github.com/SmartManoj/sparrow

Cloning into 'sparrow'...
remote: Enumerating objects: 4699, done.
remote: Counting objects: 100% (617/617), done.
remote: Compressing objects: 100% (309/309), done.
remote: Total 4699 (delta 388), reused 326 (delta 306), pack-reused 4082 (from 3)
Receiving objects: 100% (4699/4699), 11.00 MiB | 28.88 MiB/s, done.
Resolving deltas: 100% (3291/3291), done.


In [2]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y


info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: latest update on 2025-12-11, rust version 1.92.0 (ded5c06cf 2025-12-08)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
info: downloading component 'rust-std'
info: downloading component 'rustc'
info: downloading component 'rustfmt'
info: installing component 'cargo'
 10.3 MiB /  10.3 MiB (100 %)   9.1 MiB/s in  1s         
info: installing component 'clippy'
info: installing component 'rust-docs'
 20.5 MiB /  20.5 MiB (100 %)   4.5 MiB/s in  4s         
info: installing component 'rust-std'
 28.0 MiB /  28.0 MiB (100 %)   8.5 MiB/s in  3s         
info: installing component 'rustc'
 74.3 MiB /  74.3 MiB (100 %)   9.6 MiB/s in  7s         
info: installing component 'rustfmt'
info: default toolchain set to 'stable-x86_64-unknown-linux-gnu

In [3]:
 !rustup default nightly

/bin/bash: line 1: rustup: command not found


In [4]:
import os
os.environ['PATH']+=':/root/.cargo/bin'

### 1. Create input file

In [5]:
data = '''
{
  "name": "n14_h2280",
  "items": [{
    "id": 0,
    "demand": 14,
    "shape": {
      "type": "simple_polygon",
      "data": [[0,800],[125,500],[62.5,500],[200,250],[100,250],[350,0],[75,0],[75,-200],[-75,-200],[-75,0],[-350,0],[-100,250],[-200,250],[-62.5,500],[-125,500]]
    }
  }],
  "strip_height": 2280
}
'''
with open('n14_h2280.json','w') as f: f.write(data)

## Why Height 2280?

`2280` = strip height for sparrow (scaled by 1000)

**In real coordinates:**
- 2280 / 1000 = **2.280** (the height constraint)

**Target calculation:**
- Target score 0.3696 means: `side² / 14 = 0.3696`
- So target side = `sqrt(0.3696 × 14)` = **2.2747**
- Scaled: 2274.7

We use 2280 (slightly above 2275) to give sparrow room to optimize. Sparrow minimizes width for a fixed height - if height is too tight, width explodes.

**Trade-off found empirically:**

| Height | Best Width | max(w,h) | Score |
|--------|------------|----------|-------|
| 2275   | 2344       | 2344     | 0.392 |
| 2280   | 2280       | 2280     | 0.371 |
| 2300   | 2333       | 2333     | 0.389 |

Height **2280** gives the best balance where width ≈ height ≈ 2280.

### 2. Run sparrow

Parameters:
- `-t 600`: 600 seconds optimization time
- `-s 42`: random seed (try different seeds: 42, 100, 200, etc.)
- Strip height 2280 found to be optimal for N=14

In [6]:
!cd sparrow && cargo run --release --features=simd,only_final_svg -- -i ../n14_h2280.json -t 600 -s 42

    Updating crates.io index
    Updating git repository `https://github.com/JeroenGar/jagua-rs.git`
     Locking 119 packages to latest Rust 1.92.0 compatible versionsng             Fetch [========>                        ] 1 complete; 22 pending       
  Downloaded anstream v0.6.21                                              
  Downloaded anstyle-parse v0.2.7                                          
  Downloaded byteorder v1.5.0                                              
  Downloaded anstyle-query v1.1.5                                          
  Downloaded float-cmp v0.10.0 bytes: 98.1KiB                              
  Downloaded anstyle v1.0.13ng bytes: 50.3KiB                              
  Downloaded bitflags v2.10.0g bytes: 18.4KiB                              
  Downloaded web-time v1.1.0ng bytes: 20.0KiB                              
  Downloaded cfg-if v1.0.4ning bytes: 532.8KiB                             
  Downloaded matrixmultiply v0.3.10s: 522.8KiB               

In [7]:
from IPython.display import SVG, display
display(SVG('/kaggle/working/sparrow/output/final_n14_h2280.svg'))

ExpatError: not well-formed (invalid token): line 1, column 0

In [8]:
!cp /kaggle/input/santa-submission/submission.csv submission.csv

In [9]:
"""Merge sparrow solution into submission.csv if score is better."""

import json
import sys
import math
from pathlib import Path

SCALE = 1000.0

# Tree polygon vertices
TREE_VERTS = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def transform_point(x, y, tx, ty, deg):
    rad = math.radians(deg)
    c, s = math.cos(rad), math.sin(rad)
    rx = x * c - y * s
    ry = x * s + y * c
    return rx + tx, ry + ty

def calc_score(placements):
    """Calculate score from placements [(x, y, deg), ...]"""
    min_x = min_y = float('inf')
    max_x = max_y = float('-inf')

    for tx, ty, deg in placements:
        for vx, vy in TREE_VERTS:
            px, py = transform_point(vx, vy, tx, ty, deg)
            min_x = min(min_x, px)
            max_x = max(max_x, px)
            min_y = min(min_y, py)
            max_y = max(max_y, py)

    width = max_x - min_x
    height = max_y - min_y
    side = max(width, height)
    return side ** 2 / len(placements)

def load_submission(csv_path):
    """Load submission.csv into dict {id: (x, y, deg)}"""
    data = {}
    with open(csv_path) as f:
        header = f.readline()  # skip header
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split(',')
            id_ = parts[0]
            x = float(parts[1][1:])  # remove 's' prefix
            y = float(parts[2][1:])
            deg = float(parts[3][1:])
            data[id_] = (x, y, deg)
    return data

def extract_group(data, group):
    """Extract placements for a specific group (e.g., '014')"""
    placements = []
    i = 0
    while f"{group}_{i}" in data:
        placements.append(data[f"{group}_{i}"])
        i += 1
    return placements

def load_sparrow(json_path):
    """Load placements from sparrow JSON output"""
    with open(json_path) as f:
        data = json.load(f)

    sol = data.get('solution', data)
    placements = []

    for item in sol['layout']['placed_items']:
        t = item['transformation']
        x = t['translation'][0] / SCALE
        y = t['translation'][1] / SCALE
        rot = t['rotation']
        placements.append((x, y, rot))

    return placements

def merge(submission_path, sparrow_json, group="014", output_path=None):
    """Merge sparrow solution if better than existing."""

    # Load existing submission
    data = load_submission(submission_path)

    # Get existing group score
    old_placements = extract_group(data, group)
    if not old_placements:
        print(f"Error: Group {group} not found in {submission_path}")
        return False

    old_score = calc_score(old_placements)
    print(f"Existing score for {group}: {old_score:.6f}")

    # Get new sparrow score
    new_placements = load_sparrow(sparrow_json)
    new_score = calc_score(new_placements)
    print(f"New sparrow score:        {new_score:.6f}")

    if new_score >= old_score:
        print(f"\nNo improvement. Keeping existing solution.")
        return False

    improvement = old_score - new_score
    print(f"\nImprovement: {improvement:.6f} ({improvement/old_score*100:.2f}%)")

    # Merge: replace group entries
    for i, (x, y, deg) in enumerate(new_placements):
        data[f"{group}_{i}"] = (x, y, deg)

    # Write output
    out_path = output_path or submission_path

    # Sort keys properly (by group, then index)
    def sort_key(k):
        parts = k.split('_')
        return (parts[0], int(parts[1]))

    sorted_keys = sorted(data.keys(), key=sort_key)

    with open(out_path, 'w') as f:
        f.write("id,x,y,deg\n")
        for k in sorted_keys:
            x, y, deg = data[k]
            f.write(f"{k},s{x:.17f},s{y:.17f},s{deg:.17f}\n")

    print(f"Merged! Saved to {out_path}")

merge('submission.csv', 'sparrow/output/final_n14_h2280.json', "014")

Existing score for 014: 0.371478


FileNotFoundError: [Errno 2] No such file or directory: 'sparrow/output/final_n14_h2280.json'

In [ ]:
!cp -r /kaggle/working/sparrow/output output
!rm -r /kaggle/working/sparrow